In [ ]:
#resources 
#https://towardsdatascience.com/building-an-image-color-analyzer-using-python-12de6b0acf74

In [16]:
from collections import Counter
from sklearn.cluster import KMeans
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import cv2
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import colorsys
import os
from fake_useragent import UserAgent

from time import sleep
import json

import contractions
import spacy
#from spacy.tokenizer import Tokenizer
import en_core_web_sm
import nltk
#from nltk.stem import WordNetLemmatizer
from nltk import bigrams, trigrams
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import string
import regex as re
import collections
from lxml import etree

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
ua = UserAgent()
userAgent = ua.random

In [ ]:
def rgb_to_hex(rgb_color):
    hex_color = "#"
    for i in rgb_color:
        i = int(i)
        hex_color += ("{:02x}".format(i))
    return hex_color

def prep_image(raw_img):
    modified_img = cv2.resize(raw_img, (900, 600), interpolation = cv2.INTER_AREA)
    modified_img = modified_img.reshape(modified_img.shape[0]*modified_img.shape[1], 3)
    return modified_img

def color_analysis(img, url, k=5):
    
    clf = KMeans(n_clusters = k)
    color_labels = clf.fit_predict(img)
    center_colors = clf.cluster_centers_    
    counts = Counter(color_labels)
    ordered_colors = [center_colors[i].round(3).tolist() for i in counts.keys()]
    hex_colors = [rgb_to_hex(ordered_colors[i]) for i in counts.keys()]    
    plt.figure(figsize = (12, 8))
    plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)
    plt.title('color proportions from {u}'.format(u=url))
    plt.savefig('./plots/pie-' + url.replace('.', '-') + '.png')
    
    
    
    color_array = np.array(list(counts.values()))
    color_proportion = (color_array/sum(color_array)).round(3).tolist()
 
    
    color_dict = {}
    
    color_dict['centroids_rgb'] = ordered_colors
    color_dict['centroids_hex'] = hex_colors
    color_dict['color_counts'] = color_array.tolist()
    color_dict['color_proportion'] = color_proportion
    
    hsv = [list(colorsys.rgb_to_hsv(c[0], c[1], c[2])) for c in color_dict['centroids_rgb']]
    color_dict['centroids_hsv']=hsv
    
    #print(hex_colors)
    return color_dict 

In [ ]:
def get_paragraph_text(url, hdr):
    
    try:
        
        req = Request(url,headers=hdr)
        page = urlopen(req)
    
    except:
        
        print("http failed, trying https:")
        
        try:
            url = url.replace('http:', 'https:')
            req = Request(url,headers=hdr)
            page = urlopen(req)
        
        except Exception as e:
            print("substitution of https also failed. Error is {}".format(e))
            
    else:
   
        soup = BeautifulSoup(page, 'html.parser')

        p=soup.find_all('p')
        text = ' '.join(t.get_text() for t in p)

        return text

class TextPreprocessor(object):
    def __init__(self):
        """
        TBD
        """

    def preprocess_df(self, df):
        return df.apply(self._preprocess_text)

    def preprocess_text(self, text):
        doc = nlp(text)
        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)

        return self._lemmatize(removed_stop_words)

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]

    def _lemmatize(self, doc):
        
        lemma_text = ' '.join([t.lemma_ for t in doc])
        
        return re.sub('\s+',' ',lemma_text)
        


In [ ]:
urls = ['www.uber.com', 'www.lyft.com', 'www.doordash.com']
#urls = ['www.lyft.com']
#check out https://github.com/robinreni96/Font_Recognition-DeepFont

In [ ]:
options = webdriver.ChromeOptions()
options.headless = True
options.add_argument(f'user-agent={userAgent}')
    
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()
url_errors = {}
urls_color_dicts = []

#if the json already exists from a prior code run, delete it
if os.path.exists('site_screenshot_colors.json'):
    os.remove('site_screenshot_colors.json')

for url in urls:
    
    try:
        
        sleep(2)
        driver.get('https://' + url)
        sleep(2)


        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment

        el = driver.find_element(By.TAG_NAME, value='body')
        el.screenshot('./site-screenshots/'+ url.replace('.', '-') + '.png')

        image = cv2.imread('./site-screenshots/'+ url.replace('.', '-') + '.png')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)

        modified_image = prep_image(image)
        color_dict = color_analysis(modified_image, url)
        color_dict['url'] = url
        color_dict['screenshot'] = url.replace('.', '-') + '.png'
        
        
        
        with open('site_screenshot_colors.json', 'a') as f:
            f.write(json.dumps(color_dict))
            f.write('\n')

                                      

        urls_color_dicts.append(color_dict)
        
        #font family is probably not reliable
        print(el.value_of_css_property("font-family"))
        print(el.value_of_css_property("background-color"))
        print(el.value_of_css_property("color"))
        print(el.value_of_css_property("font-family"))
        print(el.value_of_css_property("font-size"))
        
        
        
                
    
    except Exception as e:
        
        print("url {url} error {e}".format(url=url, e=e))
        
        url_errors[url] = e

#driver.quit()
f.close()

print("end...")

In [ ]:
urls_color_dicts

In [ ]:
bigram_counts.most_common(50)



In [ ]:
print(soup.prettify())

In [ ]:
#element = driver.find_element_by_tag_name('title')
element = driver.find_element(By.TAG_NAME, value='title')
"/html/body"
properties = driver.execute_script('return window.getComputedStyle(arguments[0], null);', element)
for property in properties:
    print(property, element.value_of_css_property(property))

In [ ]:
#element = driver.find_element(By.XPATH, "/html/body")
#element = driver.find_element(By.TAG_NAME, value='title')

print(el.value_of_css_property("background-color"))
print(el.value_of_css_property("color"))
print(el.value_of_css_property("font-family"))
print(el.value_of_css_property("font-size"))

In [ ]:
driver.find_elements(By.CSS_SELECTOR,"@media")

In [ ]:
element = driver.find_element(By.XPATH, "/html/body")
properties = driver.execute_script('return window.getComputedStyle(arguments[0], null);', element)
for property in properties:
    print(property, element.value_of_css_property(property))

In [ ]:
#are the rgb values, cluster centroids, cluster centroid proportions, or hsv values predictive of the type of business

In [ ]:
import requests
#https://www.bcorporation.net/en-us/find-a-b-corp/search?query=food

body = {'query':'rune'} # <-- use 'query' not `Search'

con = requests.post('http://services.runescape.com/m=itemdb_rs/results#main-search', data=body)
print (con.content) # <-- print .content not .text


In [ ]:
#r = requests.get('http://www.pharmacy295.gr/el/products/autocomplete.json',data = payload ,headers = headers)
r = requests.get('https://www.bcorporation.net/en-us/find-a-b-corp/search?query=food/autocomplete.json',data = payload ,headers = headers)

In [ ]:
(r.json())

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'}

payload = {
    'query':'test',

}
r = requests.get('http://www.pharmacy295.gr/products',data = payload ,headers = headers)

soup = BeautifulSoup(r.text,'lxml')
products = soup.findAll('span', {'class':'name'})
print(products)

In [4]:
import requests
import urllib
#import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession, AsyncHTMLSession


In [ ]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)


In [ ]:
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.co.uk/search?q=" + query)

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links


In [ ]:
response = get_source("https://www.google.co.uk/search?q=" + query)

In [9]:
import asyncio
if asyncio.get_event_loop().is_running(): # Only patch if needed (i.e. running in Notebook, Spyder, etc)
    import nest_asyncio
    nest_asyncio.apply()

In [11]:
# session = HTMLSession()
# r = session.get('https://www.bcorporation.net/en-us/find-a-b-corp/search?q=food')

# #r = session.get('http://www.yourjspage.com')

# r.html.render()

asession = AsyncHTMLSession()
r = await asession.get('https://www.bcorporation.net/en-us/find-a-b-corp/search?q=food')
await r.html.arender()
resp=r.html.raw_html

[INFO] Starting Chromium download.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86.8M/86.8M [00:18<00:00, 4.81Mb/s]
[INFO] Beginning extraction
[INFO] Chromium extracted to: /Users/amywinecoff/Library/Application Support/pyppeteer/local-chromium/588429


In [13]:
r.html.arender().

<coroutine object HTML.arender at 0x7fb32a6eca40>

In [ ]:
query="food"
response = get_source("https://www.bcorporation.net/en-us/find-a-b-corp/search?q=" + query)

session = HTMLSession()

response.html.render()

In [ ]:
links = list(response.html.absolute_links)
def get_results(query):
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.bcorporation.net/en-us/find-a-b-corp/search?q=" + query)
    
    return response


In [ ]:
response = get_results("food")

In [17]:
urlpage = 'https://www.bcorporation.net/sitemap/sitemap-0.xml' 
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')

In [22]:
#xmlDict = {}

r = requests.get("https://www.bcorporation.net/sitemap/sitemap-0.xml")
xml = r.text

soup = BeautifulSoup(xml)
sitemapTags = soup.find_all("url")

In [25]:
root = etree.fromstring(r.content)

In [37]:
t=sitemap

In [47]:
len(urls)

9372

In [45]:
#Save this part
r = requests.get("https://www.bcorporation.net/sitemap/sitemap-0.xml")
root = etree.fromstring(r.content)

urls=[]
for sitemap in root:
    children = sitemap.getchildren()
    #xmlDict[children[0].text] = children[1].text
    if 'find-a-b-corp/company/' in children[0].text:
        urls.append(children[0].text)
#print (xmlDict)

In [ ]:
print(response.json())

In [ ]:
https://www.bcorporation.net/sitemap/sitemap-0.xml